In [2]:
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import os
import time
from datetime import datetime as dt
data_dir = "/Users/vigadam/Documents/github/media_network/data/"

## 24.hu

In [43]:
page_types = [
    "https://24.hu/tech/",
    "https://24.hu/szorakozas/",
    "https://24.hu/kultura/",
    "https://24.hu/fn/uzleti-tippek/",
    "https://24.hu/fn/gazdasag/",
    "https://24.hu/kulfold/",
    "https://24.hu/kozelet/",
    "https://24.hu/belfold/",
    "https://24.hu/tudomany/",
    "https://24.hu/kistotal/",
    "https://24.hu/velemeny/",
    "https://24.hu/szorakozas/",
    "https://24.hu/otthon/",
    "https://24.hu/elet-stilus/",
    "https://24.hu/sport/"
]

In [47]:
max_page = 100000
for page_type in page_types:
    articles_page_type = []
    boolean = True
    print(page_type)
    while boolean:
        for page_number in tqdm(range(1, max_page)):
            page_link = page_type + f"page/{page_number}"

            try:
                response = requests.get(page_link)
            except TimeoutError or ConnectionError:
                print(page_link)
                pass

            soup = BeautifulSoup(response.text, "html.parser")
            articles = soup.find_all("a", class_="m-articleWidget__linkImgWrap")
            articles_page_type = articles_page_type + [
                article.get("href") for article in articles
            ]

            if soup.find(attrs={"class": "next page-numbers"}) == None:
                boolean = False
                break
    df = pd.DataFrame(articles_long).drop_duplicates()
    df.to_pickle("data/links/24/24_{page_type}.pkl".format(page_type=page_type))

https://24.hu/szorakozas/


KeyboardInterrupt: 

In [122]:
link_list = os.listdir(data_dir + "links/24")

In [123]:
df = pd.DataFrame()
for cat in link_list:
    df = pd.concat([df, pd.read_pickle(data_dir + "links/24/" + cat)])

In [124]:
df = df.rename(columns={0: "link"}).loc[
    lambda x: x["link"]
    != "/?utm_source=24.hu&utm_medium=banner&utm_campaign=24.hu_right_side"
].drop_duplicates(subset=["link"]).reset_index(drop=True)
df["rovat"] = df["link"].str.split("/").apply(lambda x: x[3])

In [125]:
df = df.loc[
    df["rovat"] != "?utm_source=24.hu&utm_medium=banner&utm_campaign=24.hu_right_side"
].reset_index(drop=True)

In [126]:
df["date"] = df["link"].str.split("-").apply(lambda x: x[0]).str.split("/").apply(
    lambda x: "-".join(x[-4:-1])
)  # .apply(lambda x: dt.strptime("/".join(x[-4:-1]), "%Y/%m/%d")8

In [127]:
df.loc[df["rovat"] == "fn", "date"] = (
    df.loc[df["rovat"] == "fn", "link"].str.split("/").apply(lambda x: "-".join(x[5:8]))
)

In [128]:
df.loc[df["date"] == "https:--24.hu", "date"] = (
    df.loc[df["date"] == "https:--24.hu", "link"]
    .str.split("/")
    .apply(lambda x: "-".join(x[4:7]))
)

In [129]:
df["year"] = df["date"].apply(lambda x: x[0:4])

In [130]:
df.loc[~df["year"].apply(str.isdigit), "date"] = (
    df.loc[~df["year"].apply(str.isdigit), "link"]
    .str.split("/")
    .apply(lambda x: "-".join(x[4:7]))
)

In [131]:
df["year"] = df["date"].apply(lambda x: x[0:4])

df["month"]= df["date"].apply(lambda x: x[5:7])

In [135]:
df.to_csv(data_dir + "links/24_links_all.csv")

In [119]:
df

,link,rovat,date,year,month
0,https://24.hu/kozelet/2021/01/13/nem-volt-hete...,kozelet,2021-01-13,2021,1
1,https://24.hu/kozelet/2021/01/12/jobbik-szabo-...,kozelet,2021-01-12,2021,1
2,https://24.hu/kozelet/2021/01/12/palkovics-las...,kozelet,2021-01-12,2021,1
3,https://24.hu/kozelet/2021/01/10/gyarmati-andr...,kozelet,2021-01-10,2021,1
4,https://24.hu/kozelet/2021/01/10/szegyen-szegy...,kozelet,2021-01-10,2021,1
...,...,...,...,...,...
248085,https://24.hu/otthon/2017/05/04/sajat-kezuleg-...,otthon,2017-05-04,2017,5
248086,https://24.hu/otthon/2017/05/04/elso-latasra-s...,otthon,2017-05-04,2017,5
248087,https://24.hu/otthon/2017/05/03/oreg-garazsbol...,otthon,2017-05-03,2017,5
248088,https://24.hu/otthon/2017/05/02/hoditanak-az-o...,otthon,2017-05-02,2017,5


## 444

In [137]:
df = pd.read_pickle(data_dir +"links/444_all_links.pkl")

In [139]:
df.to_csv(data_dir +"links/444_all_links.csv")

In [ ]:
articles = []
for i in tqdm(range(6, 330)):
    page = f"https://444.hu/archivum?page={i}"
    soup = BeautifulSoup(requests.get(page).text, "html.parser")
    articles = articles + [article.get("href") for article in soup.select("h3 a")]

In [ ]:
df = df.assign(year = df[0].str[15:19]).rename(columns={0:"link"})
df["year"] = df["year"].apply(int)
df = df.loc[df["year"] == 2020].reset_index(drop=True)

In [ ]:
df["date"] = df["link"].str[15:25].apply(lambda x: dt.strptime(x, "%Y/%m/%d"))
df["month"] = df["link"].str[20:22].apply(int)

In [ ]:
df.to_pickle("data/444_all_links_2020.pkl")

___

## MNO

In [154]:
articles = []
for i in tqdm(range(1993, 2279)):
    page = f"https://magyarnemzet.hu/page/{i}/?after=2020-01-01&author&before=2020-12-31&category=0&s"
    soup = BeautifulSoup(requests.get(page).text, "html.parser")
    articles = articles + [article.get("href") for article in soup.select("h2 a")]

In [150]:
i

1993

In [155]:
df_3 = pd.DataFrame(articles, columns=["link"])

In [149]:
df_1 = df.drop_duplicates("link")

In [153]:
df_1

,link
0,https://magyarnemzet.hu/belfold/ader-janos-hat...
1,https://magyarnemzet.hu/belfold/egy-polgarmest...
2,https://magyarnemzet.hu/mindmegette/szilveszte...
3,https://magyarnemzet.hu/belfold/kariko-katalin...
4,https://magyarnemzet.hu/belfold/meghalt-mondok...
...,...
24080,https://magyarnemzet.hu/belfold/szijjarto-nem-...
24081,https://magyarnemzet.hu/kulfold/orban-viktor-l...
24082,https://magyarnemzet.hu/mozaik/hamarosan-nyit-...
24083,https://magyarnemzet.hu/belfold/a-lakossag-sze...


In [158]:
df_all = pd.concat([df_1,df_2,df_3]).reset_index(drop=True)

In [161]:
data_dir

'/Users/vigadam/Documents/github/media_network/data/'

In [162]:
df_all.to_pickle(data_dir+"links/mno_2020_links.pkl")

In [164]:
df = df_all.copy()

In [165]:
df["rovat"] = df["link"].str.split("/").apply(lambda x: x[3])

In [166]:
df

,link,rovat
0,https://magyarnemzet.hu/belfold/ader-janos-hat...,belfold
1,https://magyarnemzet.hu/belfold/egy-polgarmest...,belfold
2,https://magyarnemzet.hu/mindmegette/szilveszte...,mindmegette
3,https://magyarnemzet.hu/belfold/kariko-katalin...,belfold
4,https://magyarnemzet.hu/belfold/meghalt-mondok...,belfold
...,...,...
45500,https://magyarnemzet.hu/sport/koman-golpasszai...,sport
45501,https://magyarnemzet.hu/belfold/kislany-az-els...,belfold
45502,https://magyarnemzet.hu/gazdasag/folytatodik-a...,gazdasag
45503,https://magyarnemzet.hu/belfold/gyalogosokat-g...,belfold


In [167]:
df["felosztas"] = [round(np.random.uniform(1, 45)) for i in range(df.shape[0])]

In [170]:
df.to_csv(data_dir + "links/mno_all_links.csv")

In [172]:
df

,link,rovat,felosztas
0,https://magyarnemzet.hu/belfold/ader-janos-hat...,belfold,39
1,https://magyarnemzet.hu/belfold/egy-polgarmest...,belfold,27
2,https://magyarnemzet.hu/mindmegette/szilveszte...,mindmegette,21
3,https://magyarnemzet.hu/belfold/kariko-katalin...,belfold,10
4,https://magyarnemzet.hu/belfold/meghalt-mondok...,belfold,16
...,...,...,...
45500,https://magyarnemzet.hu/sport/koman-golpasszai...,sport,25
45501,https://magyarnemzet.hu/belfold/kislany-az-els...,belfold,40
45502,https://magyarnemzet.hu/gazdasag/folytatodik-a...,gazdasag,8
45503,https://magyarnemzet.hu/belfold/gyalogosokat-g...,belfold,42


___

## 888

In [237]:
page_types = [
    #"https://888.hu/piszkostizenketto/",
    "https://888.hu/ketharmad/",
    "https://888.hu/amerika-london-parizs/",
    "https://888.hu/big-pikcsor/",
    "https://888.hu/csak-neked-csak-most/",
    "https://888.hu/hajra-magyarok/",
    "https://888.hu/feher-ferfi/",
    "https://888.hu/pszicho/",
    "https://888.hu/kinyilott-a-pitypang/",
    # "https://888.hu/egy-soros-egy-forditott/", only one page
    "https://888.hu/censored/",
    "https://888.hu/sivalkodjatok/",
    "https://888.hu/759/",
    "https://888.hu/889/",
    "https://888.hu/888/",
    "https://888.hu/century-on/",
    "https://888.hu/szabad-vasarnap/",
    "https://888.hu/bulvar/",
    "https://888.hu/okojobb/",
    "https://888.hu/maccabi/",
]

In [244]:
articles = []
for page in page_types:
    # get list len
    resp = requests.get(page + "page/2/")

    soup = BeautifulSoup(resp.text, "html.parser")

    page_max = int(soup.select("title")[0].text.split("|")[-2].strip().split(" ")[-1])
    print(page)

    for i in tqdm(range(1, page_max)):
        try:
            resp = requests.get(page + "page/{i}/".format(i=i))
            soup = BeautifulSoup(resp.text, "html.parser")
            articles = articles + [
                i.get("href")
                for i in soup.find("div", class_="main-content").select("a")
            ]
        except Exception as e:
            print(e)
            print(page + "page/{i}/".format(i=i))

https://888.hu/ketharmad/



https://888.hu/amerika-london-parizs/


('Connection aborted.', OSError(50, 'Network is down'))
https://888.hu/amerika-london-parizs/page/783/

https://888.hu/big-pikcsor/



https://888.hu/csak-neked-csak-most/



https://888.hu/hajra-magyarok/


...


https://888.hu/century-on/



https://888.hu/szabad-vasarnap/



https://888.hu/bulvar/



https://888.hu/okojobb/



https://888.hu/maccabi/


In [245]:
"https://888.hu/amerika-london-parizs/page/783/"
"https://888.hu/kinyilott-a-pitypang/page/151/"

'https://888.hu/amerika-london-parizs/page/783/'

In [250]:
resp = requests.get("https://888.hu/amerika-london-parizs/page/783/")
soup = BeautifulSoup(resp.text, "html.parser")
articles = articles + [
    i.get("href") for i in soup.find("div", class_="main-content").select("a")
]

In [232]:
page + "page/{i}/".format(i=i)

'https://888.hu/ketharmad/page/371/'

In [252]:
nyolc = pd.DataFrame(articles, columns=["link"])

nyolc["isdigit"] = (
    nyolc["link"]
    .str.split("/")
    .apply(lambda x: x[-2])
    .str.split("-")
    .apply(lambda x: x[-1])
    .apply(str.isdigit)
)

nyolc = nyolc.loc[nyolc["isdigit"]]

nyolc["link"].str.split("/").apply(lambda x: x[-3]) != "page"

nyolc["page"] = nyolc["link"].str.split("/").apply(lambda x: x[-3]) != "page"

nyolc = nyolc.loc[nyolc["page"], "link"].to_frame().reset_index(drop=True)

nyolc["rovat"] = nyolc["link"].str.split("/").apply(lambda x: x[3])

In [255]:
nyolc["rovat"].value_counts()

amerika-london-parizs    32171
ketharmad                19899
kinyilott-a-pitypang      4813
hajra-magyarok            3834
759                       2767
szabad-vasarnap           2764
bulvar                    1617
888                        911
century-on                 789
okojobb                    513
feher-ferfi                432
sivalkodjatok              367
csak-neked-csak-most       357
889                        234
pszicho                    147
censored                   119
big-pikcsor                 68
maccabi                     28
gazemberek                   4
Name: rovat, dtype: int64

In [254]:
df.to_csv("data/888_all_links.csv")

In [111]:
with open("file.html", "w") as file:
    file.write(df_5.loc[1363,"soup"])

In [3]:
df = pd.read_csv("data/links/888_all_links.csv")

In [23]:
df["felosztas"] = [round(np.random.uniform(1, 30)) for i in range(df.shape[0])]

In [25]:
df["felosztas"].value_counts()

19    2572
14    2550
28    2544
3     2534
9     2520
23    2520
29    2512
8     2511
6     2507
16    2501
4     2500
18    2486
26    2486
2     2485
25    2485
12    2482
27    2469
7     2449
17    2444
24    2440
22    2438
11    2436
5     2434
20    2432
13    2432
15    2417
21    2415
10    2413
30    1226
1     1194
Name: felosztas, dtype: int64

In [27]:
df.to_csv("data/888_all_links.csv")